In [203]:
import numpy as np
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
def plotTSNE(tsne_results, Y_test):
    plt.figure(figsize=(12,12))
    label = Y_test
    colors = ['blue', 'red']
    plt.scatter(x=tsne_results[:,0], y=tsne_results[:,1], c=label, cmap=matplotlib.colors.ListedColormap(colors), s=3)

In [ ]:
X_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[:6000]
X_val_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[6000:8000]
X_test_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[8000:]

X_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[:6000]
X_val_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[6000:8000]
X_test_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[8000:]

X_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[:6000]
X_val_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[6000:8000]
X_test_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[8000:]

X_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[:6000]
X_val_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[6000:8000]
X_test_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[8000:]

X_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[:6000]
X_val_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[6000:8000]
X_test_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[8000:]

X_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[:6000]
X_val_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[6000:8000]
X_test_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[8000:]

Y = np.asarray(pd.read_csv("data/diabetes/Y.csv", header=None)).reshape(-1).astype(int)
Y_val = np.asarray(pd.read_csv("data/diabetes/Y_val.csv", header=None)).reshape(-1).astype(int)
Y_test = np.asarray(pd.read_csv("data/diabetes/Y_test.csv", header=None)).reshape(-1).astype(int)

In [389]:
Y2 = np.hstack([Y, Y_val])

### Gradient Boosted Decision Tree Classifier
Use grid-search and cross-validation to find optimal parameters. Note that we merged the training and validation sets into X2 and Y2, since we do 5-fold cross-validation.<br>
In this case we obtained the best results with the concatenated diagnoses.

In [397]:
X = X_lda_conc
X_val = X_val_lda_conc
X_test = X_test_lda_conc
X2 = np.vstack([X, X_val])

In [241]:
import lightgbm as lgbm
param_grid = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [11, 13, 14, 15],
    'reg_alpha': [0.6, 0.7, 0.8, 0.9],
    'min_data_in_leaf': [100, 110, 120, 130, 140, 150, 160, 170, 180],
    'max_bin': [300, 350, 400, 450, 500],
    'feature_fraction': [.2, .4, .5, .7, .8]
    }

lgb_estimator = lgbm.LGBMClassifier(objective='binary', 
                                    class_weight='balanced',
                                    eval_metric='f1',
                                    jobs=-1)
gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2_pca, y=Y2)
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=6, scoring='f1')

In [ ]:
lgb_model = gsearch.fit(X=X2, y=Y2)
print(lgb_model.best_params_, lgb_model.best_score_)

#### Optimal classifier for LDA-encoded and concatenated diagnoses

In [398]:
lgb_estimator = lgbm.LGBMClassifier(boosting_type='gbdt',
                                    min_data_in_leaf=110, \
                                    num_leaves=15, 
                                    max_bin=350, \
                                    objective='binary', \
                                    class_weight='balanced',
                                    feature_fraction=.41,
                                    num_boost_round=1000,
                                    reg_alpha=1.3
                                   )
lgb_estimator = lgb_estimator.fit(X, Y,
                      eval_set=(X_val, Y_val), 
                      early_stopping_rounds=5, 
                      eval_metric='f1', verbose=0, 
                      callbacks = [lgbm.reset_parameter(learning_rate = np.linspace(0.12, 0.001, 20).tolist()+[0.001]*980)])

C:\Users\Rafael\anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [399]:
pred = lgb_estimator.predict(X_test).astype(int)
print("accuracy   :", accuracy_score(Y_test, pred))
print("f1         :", f1_score(Y_test, pred))
print("AUROC      :", roc_auc_score(Y_test, pred))
print("prediction :", pred[:30])
print("confidence :", (np.round(np.max(lgb_estimator.predict_proba(X_test)[:30], axis=1), 1)*10).astype(int))
print("true       :", Y_test[:30])

accuracy   : 0.6365
f1         : 0.5731062830299473
AUROC      : 0.632878720285514
prediction : [1 0 0 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1]
confidence : [6 5 7 6 6 6 6 7 5 6 5 5 6 6 6 5 7 5 6 6 6 5 6 6 6 5 6 6 7 6]
true       : [0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0]


Unfortunately, this classifier only performs slightly better than random.

Looking at the feature importances reveals that the features that were one-hot encoded were attributed the least importance in the dataset. In fact, they're completely ignored by the classifier (importance 0). <br>
The most useful features are the number of inpatient, followed by the number of lab procedures. It is also worth noting that most of the features encoding the diagnoses (features 29 through 59) are among the important features.

In [239]:
print(sorted(list(zip(lgb_estimator.feature_importances_.tolist(), range(60))), reverse=True))

[(774, 20), (760, 15), (684, 7), (562, 17), (522, 21), (468, 38), (436, 3), (389, 2), (387, 49), (367, 30), (337, 0), (336, 40), (313, 5), (300, 36), (289, 31), (286, 39), (280, 54), (278, 34), (277, 47), (271, 10), (262, 26), (244, 41), (240, 18), (232, 6), (226, 13), (220, 52), (220, 37), (218, 14), (217, 32), (215, 56), (207, 33), (201, 59), (201, 4), (186, 57), (183, 51), (177, 42), (171, 44), (170, 58), (163, 19), (161, 35), (151, 28), (150, 50), (128, 16), (127, 46), (123, 45), (123, 29), (121, 48), (119, 55), (118, 53), (105, 43), (105, 27), (79, 25), (49, 1), (44, 23), (18, 22), (8, 8), (2, 24), (0, 12), (0, 11), (0, 9)]


### SVM

In [302]:
from sklearn.svm import SVC

In [455]:
X = X_lda_mean
X_val = X_val_lda_mean
X_test = X_test_lda_mean
X2 = np.vstack([X, X_val])

In [408]:
param_grid = {
    'C': [0.01, 0.1, 1, 2, 5, 15, 30],
    'kernel': ['linear', 'rbf'],
    'gamma': ['auto'],
    'shrinking': [True, False],
    'decision_function_shape': ['ovo', 'ovr']
    }

clf = SVC(class_weight='balanced')

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [ ]:
clf_model = gsearch_clf.fit(X=X2, y=Y2)
print(clf_model.best_params_, clf_model.best_score_)

In [470]:
clf = SVC(class_weight='balanced', gamma='auto', C=0.05, kernel='rbf')
clf.fit(X2, Y2)

SVC(C=0.05, break_ties=False, cache_size=200, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='auto',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [471]:
pred = clf.predict(X_test).astype(int)
print("accuracy   :", accuracy_score(Y_test, pred))
print("f1         :", f1_score(Y_test, pred))
print("AUROC      :", roc_auc_score(Y_test, pred))
print("prediction :", pred[:30])
print("true       :", Y_test[:30])

accuracy   : 0.6165
f1         : 0.5764770844837106
AUROC      : 0.6236617837728843
prediction : [1 1 0 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 1 1]
true       : [0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0]


Other methods like adaboost or knn did not even reach an f1-score of .5. 